In [ ]:
# conda: gridTools

In [ ]:
# This is the same as Example#4 except that it uses dask for
# array manipulation to keep the memory footprint down.

# NOTE: THIS EXAMPLE IS NOT COMPLETE!

# In this example, we recreate the polar sterographic IBCAO grid.
# Part A: outside the gridtools library
# Part B: using the gridtools library
# IBCAO grid details
# Cartesian grid
#   dx: 2500 meters
#   dy: 2500 meters
# Grid extent:
#   X: -2902500 to 2902500
#   Y: -2902500 to 2902500
#
# Projection
#   Stereographic
#   lat_0=90.0 (latitude of projection center)
#   lat_ts=75.0 (latitude where scale is not distorted)
#   lon_0 (not specified, at lat_0=90 it can be any value, proj defaults to 0.0)

# For proper orientation for plotting to match the IBCAO technical manual
# please use lon_0=0.0 for a plotting parameter.

# Example 5 is a python script that is similar to this notebook.
# Example 5a is the same example but using a slightly different earth radius.

# Part A: Compute the grid ourselves

In [ ]:
import numpy as np
import xarray as xr
import dask.array as da

In [ ]:
# IBCAO
# Working in cartesian coordinates, all values are in meters
# NOTE: To create a true MOM6 supergrid, the cell spacing
# is half the length of a full grid cell.
dx = 2500. / 2.
dy = 2500. / 2.
x = da.arange(-2902500., 2902500. + dx, dx, dtype=np.float32, chunks = (100))
y = da.arange(-2902500., 2902500. + dy, dy, dtype=np.float32, chunks = (100))

In [ ]:
# This is the grid size
x.shape, y.shape

In [ ]:
# Create cartesian grid (y, x)
# Confirm grid points and x, y coordinates

In [ ]:
xx, yy = da.meshgrid(x, y)

In [ ]:
# From the command line we can convert single points
# gdaltransform -s_srs "+ellps=WGS84 +proj=stere +lat_0=90 +lat_ts=75" -t_srs EPSG:4326
# The pyproj library provides for 2D array transformation of coordinates.

from pyproj import CRS, Transformer

PROJSTRING = "+proj=stere +lat_0=90 +lat_ts=75 +ellps=WGS84"

# create the coordinate reference system
crs = CRS.from_proj4(PROJSTRING)
# create the projection from lon/lat to x/y
proj = Transformer.from_crs(crs.geodetic_crs, crs)

# compute the lon/lat
# ORIGINAL
#lon, lat = proj.transform(yy, xx, direction='INVERSE')

# DASK
def gufoo(xx, yy):
    return proj.transform(xx, yy, direction='INVERSE')
lon, lat = da.apply_gufunc(gufoo, "(j),(i)->(j),(i)", xx, yy, vectorize=True, allow_rechunk=True)

In [ ]:
# Confirm we have the correct grid points and lat lon values
# Expected answers:
# -2902500.0 -2902500.0 53.8170746379705 -45.0
# 2902500.0 2902500.0 53.8170746379705 135.0
# REF: https://www.ngdc.noaa.gov/mgg/bathymetry/arctic/IBCAO_TechnicalReference.PDF
print(float(yy[0,0]), float(xx[0,0]), float(lat[0,0]), float(lon[0,0]))
print(float(yy[y.shape[0]-1, x.shape[0]-1]), float(xx[y.shape[0]-1, x.shape[0]-1]), float(lat[y.shape[0]-1, x.shape[0]-1]), float(lon[y.shape[0]-1, x.shape[0]-1]))

In [ ]:
from gridtools.gridutils import GridUtils
%matplotlib inline

In [ ]:
grd = GridUtils()

In [ ]:
grd.clearGrid()

In [ ]:
# Define IBCAO grid for gridtools library
grd.setGridParameters({
    'projection': {
        'name': "Stereographic",
        'ellps': 'WGS84',
        'lon_0': 0.0,
        'lat_0': 90.0,
        'lat_ts': 75.0,
        'dx': 5000.0,
        'dy': 5000.0,
        'dxUnits': 'meters',
        'dxUnits': 'meters'
    }
})
grd.grid['x'] = (('nyp','nxp'), lon)
grd.grid['y'] = (('nyp','nxp'), lat)

In [ ]:
grd.grid

In [ ]:
grd.setPlotParameters(
    {
        'figsize': (8,8),
        'projection': {
            'name': 'Stereographic',    
            'lon_0': 0.0,
            'lat_0': 90.0
        },
        'extent': [-180, 180, 60, 90],
        'iLinewidth': 1,
        'jLinewidth': 1,
        'showGrid': True,
        'title': 'Stereographic: IBCAO',
        'iColor': 'y',
        'jColor': 'k'
    }
)

In [ ]:
(figure, axes) = grd.plotGrid()

In [ ]:
figure

In [ ]:
# Compute grid metrics
grd.computeGridMetricsSpherical()

In [ ]:
grd.grid

# Part B: Use the gridtools library

In [ ]:
# Clear the grid and plot parameters
grd.clearGrid()

In [ ]:
# From the technical documentation, we know:
#  * the center point,
#  * the grid extent
#  * the grid resolution
#  * the projection: standard ellipse and polar projection
#  * the latitude of true scale
#
# The full size of the grid is 2902500 + 2902500 meters (5805000 meters)
# North Pole: lat_0 = 90.0
# Latitude of true scale: lat_ts = 75.0
grd.setGridParameters({
    'projection': {
        'name': "Stereographic",
        'ellps': 'WGS84',
        'lon_0': 0.0,
        'lat_0': 90.0,
        'lat_ts': 75.0,
    },
    'centerX': 0.0,
    'centerY': 90.0,
    'cneterUnits': 'degrees',
    'dx': 5805000.0,
    'dy': 5805000.0,
    'dxUnits': 'meters',
    'dyUnits': 'meters',
    'gridResolution': 2500.0,
    'gridResolutionUnits': 'meters',
    'tilt': 0.0,
    'gridMode': 2,
    'gridType': 'MOM6',
    'ensureEvenI': True,
    'ensureEvenJ': True
})

In [ ]:
grd.makeGrid()

In [ ]:
grd.grid

In [ ]:
grd.setPlotParameters(
    {
        'figsize': (8,8),
        'projection': {
            'name': 'Stereographic',    
            'lon_0': 0.0,
            'lat_0': 90.0
        },
        'extent': [-180, 180, 60, 90],
        'iLinewidth': 1,
        'jLinewidth': 1,
        'showGrid': True,
        'title': 'Stereographic: IBCAO',
        'iColor': 'y',
        'jColor': 'k'
    }
)

In [ ]:
(figure, axes) = grd.plotGrid()

In [ ]:
figure